In [2]:
import ee
ee.Authenticate()
ee.Initialize()


In [3]:
# Define the ROIs (already provided)
roi_for_brightness = ee.Geometry.Polygon(
    [[163.05418845709113, -77.62217141675877], 
    [163.04800864752082, -77.61908009873933], 
    [163.0795943408802, -77.61422077772508], 
    [163.08302756841925, -77.61569349746868],
    [163.09813376959113, -77.60950691594756], 
    [163.1407057910755, -77.60567560295046], 
    [163.15924521978644, -77.60228539245952], 
    [163.1736647754505, -77.60140083961204], 
    [163.17435142095832, -77.5996315475443],
    [163.18259116705207, -77.59786200695194],
    [163.19769736822394, -77.60125340809783], 
    [163.21417686041144, -77.60184312380088], 
    [163.2505690723255, -77.59874680831578], 
    [163.25812217291144, -77.60036878277327],
    [163.25400229986457, -77.60302247238047],
    [163.25468894537238, -77.60493867830638], 
    [163.25880881841925, -77.60670722499943], 
    [163.2334029346302, -77.60788611813062], 
    [163.21967002447394, -77.60655985559598],
    [163.20799705084113, -77.60980158403946],
    [163.21280356939582, -77.61156944773495], 
    [163.19357749517707, -77.61716604483516], 
    [163.18121787603644, -77.6155462332516], 
    [163.15237876470832, -77.61687154915111],
    [163.14894553716925, -77.62025783330262],
    [163.12010642584113, -77.62231860342517], 
    [163.10637351568488, -77.62217141675877], 
    [163.0795943408802, -77.61893287416414], 
    [163.06448813970832, -77.61849119009841],
    [163.06723472173957, -77.62099386140082],
    [163.06448813970832, -77.6226129715892],
    [163.05418845709113, -77.62217141675877]]
)



In [4]:
# Define ROI for soil
roi_soil = ee.Geometry.Rectangle([163.078070, -77.625204, 163.07800, -77.625340])

# Pull image
soil1 = ee.Image('LANDSAT/LC08/C02/T2_TOA/LC08_055116_20231205').select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])

# Clip image down to ROI for soil
soil = soil1.clip(roi_soil)

# Reduce down to mean for endmembers
soil_mean_defined = soil.reduceRegion(ee.Reducer.mean()).values()

In [5]:
# Function to select the brightest pixels and compute 3x3 averages
def select_brightest_pixels_and_average(image, roi, n=1):
    # Step 1: Compute brightness as the sum of selected bands
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6']).reduce(ee.Reducer.sum())
    print("Step 1: Brightness calculation complete.")

    # Step 2: Sample brightness values within the region of interest
    sampled_points = brightness.sample(region=roi, scale=30, numPixels=1000)
    print("Step 2: Brightness sampled.")

    # Step 3: Sort sampled points by brightness
    sorted_points = sampled_points.sort('sum', False)
    print("Step 3: Points sorted by brightness.")

    # Step 4: Select the top N brightest points
    top_n_brightest_points = sorted_points.limit(n)
    print("Step 4: Top N brightest points selected.")

    # Step 5: Compute the mean values for a 3x3 window around each point
    def compute_mean_of_window(feature):
        coords = feature.geometry().coordinates()
        # Safeguard: Check if coordinates are valid
        valid = ee.Algorithms.IsEqual(coords.length(), 2)
        return ee.Algorithms.If(
            valid,
            # Valid coordinates: Compute mean within the buffer
            feature.set(
                image.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=ee.Geometry.Point(coords).buffer(90),
                    scale=30
                )
            ),
            # Invalid coordinates: Return the feature unchanged
            feature
        )

    means = top_n_brightest_points.map(compute_mean_of_window)
    print("Step 5: Mean values computed for 3x3 windows.")

    # Step 6: Extract the band means from the computed features
    def extract_band_means(feature):
        band_names = image.bandNames()
        values = band_names.map(lambda band: feature.get(band))
        return ee.Feature(None, {'values': values})

    features_with_values = means.map(extract_band_means)
    print("Step 6: Band means extracted.")

    # Step 7: Aggregate the results into a list of values
    aggregated_values = features_with_values.aggregate_array('values')
    print("Step 7: Aggregated values into a list.")

    return aggregated_values

# Test the function with added debugging
try:
    test_result = select_brightest_pixels_and_average(soil1, roi_for_brightness, n=1)
    print("Function executed successfully. Result:")
    print(test_result.getInfo())  # Check the output
except Exception as e:
    print(f"Error encountered: {e}")


Step 1: Brightness calculation complete.
Step 2: Brightness sampled.
Step 3: Points sorted by brightness.
Step 4: Top N brightest points selected.
Step 5: Mean values computed for 3x3 windows.
Step 6: Band means extracted.
Step 7: Aggregated values into a list.
Function executed successfully. Result:
[[None, None, None, None, None, None, None]]


In [30]:
# Function to select the brightest pixels and compute 3x3 averages
def select_brightest_pixels_and_average(image, roi, n=1):
    # Step 1: Compute brightness as the sum of selected bands
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6']).reduce(ee.Reducer.sum())
    print("Step 1: Brightness calculation complete.", brightness.getInfo())

    # Step 2: Sample brightness values within the region of interest
    sampled_points = brightness.sample(region=roi, scale=30, numPixels=1000)
    print("Step 2: Brightness sampled.")

    # Step 3: Sort sampled points by brightness
    sorted_points = sampled_points.sort('sum', False)
    print("Step 3: Points sorted by brightness.")

    # Step 4: Select the top N brightest points
    top_n_brightest_points = sorted_points.limit(n)
    print("Step 4: Top N brightest points selected.")

    # Step 5: Compute the mean values for a 3x3 window around each point
    def compute_mean_of_window(feature):
        coords = feature.geometry().coordinates()
        valid = ee.Algorithms.IsEqual(coords.length(), 2)
        return ee.Algorithms.If(
            valid,
            # Valid coordinates: Compute mean within the buffer
            feature.set(
                image.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=ee.Geometry.Point(coords).buffer(90),  # Increased buffer size
                    scale=30,
                    maxPixels=54808  # Adjust to handle edge cases with larger buffers
                )
            ),
            # Invalid coordinates: Return the feature unchanged
            feature
        )

    means = top_n_brightest_points.map(compute_mean_of_window)
    print("Step 5: Mean values computed for 3x3 windows.")

    print(means.getInfo())

    # Step 6: Extract the band means from the computed features
    def extract_band_means(feature):
        band_names = image.bandNames()
        values = band_names.map(lambda band: feature.get(band))
        return ee.Feature(None, {'values': values})

    features_with_values = means.map(extract_band_means)
    print("Step 6: Band means extracted.")
    print(features_with_values.getInfo())

    # Step 7: Aggregate the results into a list of values
    aggregated_values = features_with_values.aggregate_array('values')
    print("Step 7: Aggregated values into a list.")

    return aggregated_values

# Test the function with added debugging
try:
    test_result = select_brightest_pixels_and_average(soil1, roi_for_brightness, n=1)
    print("Function executed successfully. Result:")
    print(test_result.getInfo())  # Check the output
except Exception as e:
    print(f"Error encountered: {e}")


Step 1: Brightness calculation complete. {'type': 'Image', 'bands': [{'id': 'sum', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'PROJCS["PS         WGS84", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563]], \n    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Polar_Stereographic"], \n  PARAMETER["central_meridian", 0.0], \n  PARAMETER["latitude_of_origin", -71.0], \n  PARAMETER["scale_factor", 1.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'crs_transform': [30, 0, 238785, 0, -30, -1109985]}], 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[161.92785075166503, -74.64120616103985], [160.30380216402287, -74.17853433739408], [1

In [31]:
# Define a point of interest
roi_fortest = ee.Geometry.BBox(162.1, -77.73, 163.3, -77.59)
t = select_brightest_pixels_and_average(soil1, roi_box, n=1)

Step 1: Brightness calculation complete. {'type': 'Image', 'bands': [{'id': 'sum', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'PROJCS["PS         WGS84", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563]], \n    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Polar_Stereographic"], \n  PARAMETER["central_meridian", 0.0], \n  PARAMETER["latitude_of_origin", -71.0], \n  PARAMETER["scale_factor", 1.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'crs_transform': [30, 0, 238785, 0, -30, -1109985]}], 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[161.92785075166503, -74.64120616103985], [160.30380216402287, -74.17853433739408], [1

In [32]:
print(t.getInfo())

[[None, None, None, None, None, None, None]]


In [6]:
# Function to define endmembers
def define_endmembers(image, roi, roi_soil, n=1):
    brightest_pixel_means = select_brightest_pixels_and_average(image, roi, n)

    if ee.List(brightest_pixel_means).size().getInfo() == 0:
        print("Error: No valid brightest pixel means calculated.")
        return None

    soil_mean = soil_mean_defined
    endmembers = ee.List([brightest_pixel_means, soil_mean])
    return endmembers

In [7]:
# Function to perform spectral unmixing
def perform_unmixing(image, endmembers):
    if endmembers is None:
        return None

    brightest_pixel_means = endmembers.get(0)
    soil_mean = endmembers.get(1)
    unmixed_image = image.unmix([brightest_pixel_means, soil_mean])
    return unmixed_image.set('system:time_start', image.get('system:time_start'))

In [16]:
simple_point = ee.Geometry.Point([163.07807, -77.625204])  # Replace with a valid coordinate in your ROI


In [19]:
#define a new ROI that is potentially simpler
roi_box = ee.Geometry.Polygon(
    [[163.12698754679585, -77.6073690739641], 
    [163.16406640421772, -77.60946879016157], 
    [163.15445336710835, -77.61628124600912], 
    [163.11823281657124, -77.6140353843151],
    [163.12698754679585, -77.6073690739641]]
)

In [29]:
def test_with_simple_geometry(image, point_geometry):
    # Calculate brightness as the sum of selected bands
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6']).reduce(ee.Reducer.sum())

    # Sample brightness at the simple point
    sampled_brightness = brightness.sample(region=point_geometry, scale=30, numPixels=1)
    print("Sampled Brightness:", sampled_brightness.getInfo())

    # Compute mean values within a 3x3 window around the simple point
    buffer = point_geometry.buffer(90)  # 3x3 Landsat pixel window
    band_means = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=buffer,
        scale=30,
        maxPixels=54808
    )
    print("Band Means:", band_means.getInfo())

# Run the test with the simple point
try:
    test_with_simple_geometry(soil1, roi_for_brightness)
except Exception as e:
    print(f"Error encountered during simple geometry test: {e}")

Sampled Brightness: {'type': 'FeatureCollection', 'columns': {}, 'properties': {'band_order': ['sum']}, 'features': [{'type': 'Feature', 'geometry': None, 'id': '0', 'properties': {'sum': 0.8648965060710907}}]}
Band Means: {'B2': 0.4712624362655445, 'B3': 0.44842190014003747, 'B4': 0.4312333929459873, 'B5': 0.29466057632094433, 'B6': 0.022114755705100693, 'B7': 0.02000321287229315, 'B8': 0.8845274059586611}


In [26]:
t2 = test_with_simple_geometry(soil1, roi_box)

Sampled Brightness: {'type': 'FeatureCollection', 'columns': {}, 'properties': {'band_order': ['sum']}, 'features': [{'type': 'Feature', 'geometry': None, 'id': '0', 'properties': {'sum': 1.776140340603888}}]}
Band Means: {'B2': 0.4551902374176419, 'B3': 0.43710820639887227, 'B4': 0.42775998972499985, 'B5': 0.30599613562037625, 'B6': 0.009323718265520914, 'B7': 0.00774588683653116, 'B8': 0.8868256822503188}
